In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

import shared_utils
import branca
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Get and transform test data (Long Beach Transit, 170)

In [2]:
lbt_itp_id = 170

In [82]:
tbl.views.gtfs_schedule_fact_daily_trips() >> filter(_.calitp_itp_id == 372)

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,calitp_extracted_at,calitp_deleted_at,...,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,5533428114026904288,-8894101293156335779,873c5ce9-b4d6-495e-9c07-c093dc22902e,94bf7fe8-7d47-4f99-a4a1-630e5f08e6cb,372,0,7d10f282-0077-4845-b880-a70459fa9e76,2022-10-26,None,None,...,None,None,True,None,True,6,6,43800,45960,0.600000
1,5533428114026904288,5524187219740965339,e4f05056-e091-4abe-92ed-1c3004ae5a88,3b108a90-2b16-4dbb-9831-fb13556d15ac,372,0,7d10f282-0077-4845-b880-a70459fa9e76,2022-08-29,None,None,...,None,None,True,None,True,64,65,70200,78480,2.300000
2,5533428114026904288,-6902769983788250578,b3ccb3c4-cd62-44cf-86ee-3f49162eee75,01ba20cd-f96b-49f3-8692-984186008da6,372,0,42a0b184-bdd2-49c3-a70d-6426b366f83c,2022-05-20,None,None,...,None,None,True,None,True,2,2,49020,49500,0.133333
3,5533428114026904288,665807638275895737,e656702d-8fc2-4b58-8464-a9e703d6d9be,01ba20cd-f96b-49f3-8692-984186008da6,372,0,7d10f282-0077-4845-b880-a70459fa9e76,2022-07-19,None,None,...,None,None,True,None,True,16,16,61500,64740,0.900000
4,5533428114026904288,4104246313111234078,d18b4009-a104-442c-8243-c2084cbb935b,94bf7fe8-7d47-4f99-a4a1-630e5f08e6cb,372,0,7d10f282-0077-4845-b880-a70459fa9e76,2022-01-06,None,None,...,None,None,True,None,True,6,6,61800,64080,0.633333


In [3]:
# %%sql -o lbt_positions

# # get all vehicle positions on selected dates, for the feed with itp id 170, and url number 0
# SELECT *
# FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
# WHERE _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-01_170_0.parquet"
#     OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-11-30_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-03_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-04_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-05_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-06_170_0.parquet"
#     # OR _FILE_NAME="gs://gtfs-data/rt-processed/vehicle_positions/vp_2021-12-07_170_0.parquet"
# ORDER BY header_timestamp

#### Reformat timestamps

In [4]:
# ## set system time
# os.environ['TZ'] = 'America/Los_Angeles'
# time.tzset()
# time.tzname

In [5]:
# lbt_positions.vehicle_timestamp = lbt_positions.vehicle_timestamp.apply(convert_ts)
# lbt_positions.header_timestamp = lbt_positions.header_timestamp.apply(convert_ts)

In [6]:
# lbt_positions.to_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [7]:
lbt_positions = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')

In [8]:
pch_routes = ('171', '172', '173', '174', '175')

In [88]:
# min_date, max_date = ('2021-11-01', '2021-11-30')

# lbt_trips = (tbl.views.gtfs_schedule_fact_daily_trips()
#     >> filter(_.calitp_extracted_at <= min_date, _.calitp_deleted_at > max_date)
#     >> filter(_.calitp_itp_id == lbt_itp_id)
#     >> filter(_.service_date == max_date)
#     >> filter(_.is_in_service == True)
#     >> select(_.trip_key, _.service_date)
#     >> inner_join(_, tbl.views.gtfs_schedule_dim_trips(), on = 'trip_key')
#     >> select(_.calitp_itp_id, _.calitp_url_number, _.service_date,
#               _.trip_key, _.trip_id, _.route_id, _.direction_id,
#               _.shape_id, _.calitp_extracted_at, _.calitp_deleted_at)
#     # >> filter(_.route_id.isin(pch_routes))

#     >> collect()
#     )

In [89]:
# lbt_trips.to_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [90]:
lbt_trips = pd.read_parquet(f'{GCS_FILE_PATH}lbt_trips.parquet')

In [12]:
# stop_times = (tbl.gtfs_schedule.stop_times()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      >> arrange(_.stop_sequence)
#      >> collect()
#     )

In [13]:
# stop_times.to_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [14]:
stop_times = pd.read_parquet(f'{GCS_FILE_PATH}lbt_st.parquet')

In [15]:
# stops = (tbl.gtfs_schedule.stops()
#      >> filter(_.calitp_itp_id == 170)
#      # >> filter(_.trip_id == '9288623')
#      # >> mutate(stop_sequence = _.stop_sequence.astype(int)) ## in SQL!
#      # >> arrange(_.stop_sequence)
#      >> select(_.calitp_itp_id, _.calitp_url_number, _.stop_id,
#               _.stop_lat, _.stop_lon, _.stop_name)
#      >> collect()
#     )

# stops = gpd.GeoDataFrame(stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
#                         crs='EPSG:4326').to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [16]:
# shared_utils.utils.geoparquet_gcs_export(stops, GCS_FILE_PATH, 'lbt_stops')

In [17]:
stops = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_stops.parquet')

In [18]:
trips_positions_joined = (lbt_trips
                        # >> filter(_.route_id.isin(pch_routes))
                        >> inner_join(_, (lbt_positions >> rename(trip_id = 'vehicle_trip_id')),
                                      on= ['trip_id', 'calitp_itp_id', 'calitp_url_number'])
                       )

In [19]:
trips_positions_joined.vehicle_timestamp.max()

Timestamp('2021-12-01 11:15:59')

In [20]:
trips_positions_joined.vehicle_timestamp.min()

Timestamp('2021-11-29 15:57:13')

In [21]:
trips_positions_joined = gpd.GeoDataFrame(trips_positions_joined,
                                geometry=gpd.points_from_xy(trips_positions_joined.vehicle_position_longitude,
                                                            trips_positions_joined.vehicle_position_latitude),
                                crs=shared_utils.geography_utils.WGS84).to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [22]:
# lbt_routelines = shared_utils.geography_utils.make_routes_shapefile(['170'], ## LBT
#                                 shared_utils.geography_utils.CA_NAD83Albers)

In [23]:
# shared_utils.utils.geoparquet_gcs_export(lbt_routelines, GCS_FILE_PATH, 'lbt_routelines')

In [24]:
lbt_routelines = gpd.read_parquet(f'{GCS_FILE_PATH}lbt_routelines.parquet')

In [25]:
trips_positions_joined = trips_positions_joined >> filter(_.vehicle_timestamp.apply(lambda x: x.day == 30))

In [26]:
trips_positions_joined.head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
285,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:50:38,POINT (174524.511 -468602.082)
286,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:50:38,POINT (174524.511 -468602.082)
287,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01,2706,None,2021-11-30 17:50:18,33.78208,-118.11597,2021-11-30 17:51:08,POINT (174524.511 -468602.082)


In [27]:
stop_times.head(3)

,calitp_itp_id,calitp_url_number,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at
0,170,0,9285982,0666,1,10:41:00,10:41:00,None,0,0,None,None,0.0,1,2021-12-17
1,170,0,9288986,4003,1,17:05:00,17:05:00,None,0,1,None,None,0.0,1,2021-12-17
2,170,0,9286129,4012,1,14:20:00,14:20:00,None,0,1,None,None,0.0,1,2021-12-17


## Vehicle Positions Trip class

#### Examples

In [28]:
example_trip = (trips_positions_joined
 >> filter(_.trip_key == 1553149201202261125)
)

In [29]:
example_trip.head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,entity_id,vehicle_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry
327364,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:02:37,POINT (174550.467 -468602.675)
327365,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:02:37,POINT (174550.467 -468602.675)
327366,170,0,2021-11-30,1553149201202261125,9288623,175,0,1750012,2021-09-16,2099-01-01,2901,None,2021-11-30 17:02:12,33.78207,-118.11569,2021-11-30 17:03:07,POINT (174550.467 -468602.675)


In [30]:
example_vp_trip = rt.VehiclePositionsTrip(example_trip, lbt_routelines)

sc_called
(59, 4)


In [31]:
example_vp_trip.vehicle_positions.head(3)

,vehicle_timestamp,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
0,2021-11-30 17:02:12,2021-11-30 17:02:37,POINT (174550.467 -468602.675),74.936069,NaT,NaN,NaN,NaN,False,NaN
1,2021-11-30 17:03:13,2021-11-30 17:03:37,POINT (174199.373 -468606.652),426.501155,2021-11-30 17:02:12,74.936069,61.0,351.565087,True,5.763362
2,2021-11-30 17:04:13,2021-11-30 17:04:37,POINT (173950.148 -468607.933),676.459161,2021-11-30 17:03:13,426.501155,60.0,249.958006,True,4.165967


In [32]:
example_vp_trip.time_at_position(550) ## interpolate time at 550 meters from start of route (shape_meters in df^)

Timestamp('2021-11-30 17:03:42.644702')

In [33]:
# example_vp_trip.detailed_speed_map()

## General RT Analysis class

In [34]:
example_trips = (trips_positions_joined
 >> filter(_.route_id == '175', _.direction_id == '0') ## towards Villages at Cabrillo
 >> group_by(_.trip_key)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 >> head(6) ## afternoon/evening
)
example_trips

,trip_key,max_time
4,-5374575435373645071,2021-11-30 07:14:16
12,3082123411846948636,2021-11-30 07:54:17
0,-8296488155279914403,2021-11-30 08:38:01
1,-7508921904019176003,2021-11-30 09:15:30
14,4908653065062143760,2021-11-30 09:54:22
5,-4201736542695259394,2021-11-30 10:35:43


In [35]:
%%capture
## suppress warnings from location data inaccuracies
## generate test analysis with a small subset of LBT trips on a particular route, time, direction
test_analysis = rt.RtAnalysis(trips_positions_joined, stop_times, stops, lbt_routelines, example_trips.trip_key)

In [36]:
# %%capture
## suppress warnings from start/end of routes
all_delays = test_analysis.generate_delay_view()

gdv called
insufficient bounding points for trip -5374575435373645071, location 11480.641185357437: start/end of route?
insufficient bounding points for trip -5374575435373645071, location 991.1092912035456: start/end of route?
insufficient bounding points for trip -5374575435373645071, location 0.0: start/end of route?
insufficient bounding points for trip -5374575435373645071, location 501.0407611454837: start/end of route?


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


insufficient bounding points for trip 3082123411846948636, location 11480.641185357437: start/end of route?
insufficient bounding points for trip 3082123411846948636, location 0.0: start/end of route?
insufficient bounding points for trip -8296488155279914403, location 11480.641185357437: start/end of route?


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


insufficient bounding points for trip -8296488155279914403, location 0.0: start/end of route?
insufficient bounding points for trip -8296488155279914403, location 501.0407611454837: start/end of route?


...

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


insufficient bounding points for trip 4908653065062143760, location 0.0: start/end of route?
insufficient bounding points for trip 4908653065062143760, location 501.0407611454837: start/end of route?
5 trips processed
insufficient bounding points for trip -4201736542695259394, location 11480.641185357437: start/end of route?


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


insufficient bounding points for trip -4201736542695259394, location 0.0: start/end of route?


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [37]:
all_summary = test_analysis.generate_delay_summary()

In [38]:
all_summary.head(5)

,stop_id,geometry,stop_sequence,avg_delay,max_delay
0,1027,POINT (174124.462 -468604.478),2,0 days 00:01:26.446291666,0 days 00:04:11.371129
1,0540,POINT (173921.407 -468897.107),3,0 days 00:01:07.972914400,0 days 00:04:28.865438
2,0542,POINT (173328.363 -468647.513),4,0 days 00:00:39.873642666,0 days 00:03:58.590881
3,0543,POINT (173108.288 -468477.715),5,0 days 00:00:37.971737666,0 days 00:03:32.980943
4,0544,POINT (173001.551 -468534.279),6,0 days 00:01:56.187369166,0 days 00:03:37.006580


In [39]:
# test_analysis.map_stop_delays(how = 'average') ## to refine, add hover, legend, etc...

In [71]:
# test_analysis.map_segment_speeds(how = 'low_speeds')

In [42]:
# test_analysis.segment_speed_view

In [70]:
# test_analysis.map_stop_delays()

## Sandbox

In [44]:
trips_positions_joined >> filter(_.direction_id == '0') >> count(_.trip_id)

,trip_id,n
0,9288152,201
1,9288153,326
2,9288154,283
3,9288155,276
4,9288156,276
...,...,...
834,9296933,63
835,9296934,106
836,9296935,69
837,9297176,68


In [45]:
trips_positions_joined.vehicle_timestamp.iloc[0].day

30

In [48]:
pm_peak = (trips_positions_joined
 >> filter(_.direction_id == '0') ## towards 
 >> filter((_.vehicle_timestamp.apply(lambda x: x.hour < 20))
           & (_.vehicle_timestamp.apply(lambda x: x.hour > 16))) ## 4-8pm
 >> mutate(timestamp_day = _.vehicle_timestamp.apply(lambda x: x.day))
 # >> mutate(timestamp_hour = _.vehicle_timestamp.apply(lambda x: x.hour))
 # >> filter((_.timestamp_hour < 20)
 #           & (_.timestamp_hour > 16)) ## 4-8pm >> group_by(_.trip_id)
 >> filter(_.timestamp_day == 30)
 >> group_by(_.trip_key)
 >> summarize(max_time = _.vehicle_timestamp.max())
 >> arrange(_.max_time)
 # >> head(40) ## afternoon/evening
)
pm_peak

,trip_key,max_time
112,1452693937536679787,2021-11-30 17:00:34
183,8516868781734655921,2021-11-30 17:01:25
111,1330214420259335993,2021-11-30 17:01:52
154,6086303368573544582,2021-11-30 17:02:20
160,6798393323295676997,2021-11-30 17:03:04
...,...,...
138,4555117944670782770,2021-11-30 19:59:52
178,8047417353098991682,2021-11-30 19:59:52
103,587145373864954386,2021-11-30 19:59:54
152,5733593844652457787,2021-11-30 19:59:55


In [49]:
test_many = rt.RtAnalysis(trips_positions_joined,
                           stop_times,
                           stops,
                           lbt_routelines,
                           pm_peak.trip_key)

sc_called
(98, 4)
sc_called
(80, 4)
sc_called
(69, 4)
sc_called
(41, 4)
check location data for trip 6086303368573544582
sc_called
(34, 10)
check location data for trip 6086303368573544582
sc_called
(31, 10)
sc_called
(21, 4)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(100, 4)
sc_called
(86, 4)
sc_called
(77, 4)
sc_called
(22, 4)
sc_called
(28, 4)
sc_called
(76, 4)
sc_called
(35, 4)
sc_called
(109, 4)
sc_called
(39, 4)
sc_called
(67, 4)
sc_called
(94, 4)
sc_called
(65, 4)
sc_called
(46, 4)
sc_called
(60, 4)
check location data for trip -6507058681082383946
sc_called
(41, 10)
check location data for trip -6507058681082383946
sc_called
(35, 10)
check location data for trip -6507058681082383946
sc_called
(32, 10)
check location data for trip -6507058681082383946
sc_called
(30, 10)
sc_called
(103, 4)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(71, 4)
sc_called
(58, 4)
sc_called
(89, 4)
sc_called
(75, 4)
check location data for trip -915202318293282742
sc_called
(50, 10)
check location data for trip -915202318293282742
sc_called
(45, 10)
check location data for trip -915202318293282742
sc_called
(42, 10)


...

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(72, 4)
check location data for trip 61649734453040608
sc_called
(56, 10)
check location data for trip 61649734453040608
sc_called
(53, 10)
sc_called
(103, 4)
sc_called
(69, 4)
check location data for trip -1477640557036943499
sc_called
(53, 10)
check location data for trip -1477640557036943499
sc_called
(50, 10)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


sc_called
(49, 4)
sc_called
(42, 4)
sc_called
(63, 4)
sc_called
(65, 4)
sc_called
(89, 4)
check location data for trip 5733593844652457787
sc_called
(72, 10)
check location data for trip 5733593844652457787
sc_called
(69, 10)
sc_called
(80, 4)


/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [50]:
%%capture --no-stderr
test_many.generate_delay_view()

/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/opt/conda/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [52]:
test_many.trip_keys.nunique()

195

In [53]:
test_many.delay_view.trip_id.nunique()

193

In [54]:
test_many.delay_view.shape_id.nunique()

39

In [55]:
test_many.delay_view.head(3)

,trip_key,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,service_date,geometry
2064,-8969185497562832492,2021-11-30 17:23:32,2021-11-30 17:24:22.431133,0 days 00:00:50.431133,0647,9289269,230018,0,Southbound,2,23,202.128115,2021-11-30,POINT (169388.237 -455229.468)
8407,-8904883346845697341,2021-11-30 18:21:23,2021-11-30 18:25:48.094638,0 days 00:04:25.094638,3062,9288535,910132,0,Southbound,2,91,832.866019,2021-11-30,POINT (174171.028 -456696.680)
8589,-8206098010521959557,2021-11-30 18:39:24,2021-11-30 18:38:00.486673,-1 days +23:58:36.486673,0403,9288930,1820071,0,Southbound,2,182,456.351601,2021-11-30,POINT (167091.467 -464773.828)


In [56]:
df = (test_many.route_coverage_summary()
 >> mutate(stops_covered = _.max_stop - _.min_stop)
 >> arrange(-_.stops_covered)
 >> filter(_.stops_covered > 1)
 >> distinct(_.shape_id, _keep_all=True)
)
df.head(5)

,trip_key,trip_id,shape_id,direction_id,min_meters,min_stop,max_meters,max_stop,stops_covered
0,-8904883346845697341,9288535,910132,0,832.866019,2,25107.200799,80,78
1,-9051957043580796970,9288532,910131,0,565.808266,3,24385.983516,80,77
2,-1015450396633306145,9288652,1730123,0,785.141960,2,31511.120110,79,77
3,-8433175554567067531,9288223,1920167,0,1029.767210,3,26019.972326,77,74
4,-7267647641774038198,9288533,930147,0,832.866019,2,26047.441803,76,74


In [69]:
# test_many.map_stop_delays() ## broken/inf recursion (delay view check failing?)

In [72]:
# test_many.map_segment_speeds(how = 'low_speeds')

In [59]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [60]:
m = test_many._generate_segment_map(how = 'low_speeds', colorscale = colorscale)

In [68]:
# m

In [62]:
test_delay = test_many.delay_view.copy()

### Sandbox

In [63]:
#TODO rewrite to not modify attributes!!

endpoint_delay_sum = (test_delay
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.max())
 >> ungroup()
 >> mutate(total_delay = _.delay.sum())
)

In [64]:
starts = (test_delay
 >> group_by(_.trip_id)
 >> filter(_.stop_sequence == _.stop_sequence.min())
 >> ungroup()
 # >> mutate(total_delay = _.delay.sum())
 >> select(_.start_time == _.actual_time, _.trip_id, _.start_meters == _.shape_meters)
)

In [65]:
speed_and_delay = endpoint_delay_sum >> inner_join(_, starts, on='trip_id')

speed_and_delay['seconds'] = speed_and_delay.apply(
    lambda x: (x.actual_time - x.start_time).seconds, axis=1)

speed_and_delay['trip_speed'] = (speed_and_delay.shape_meters - speed_and_delay.start_meters) / speed_and_delay.seconds

speed_and_delay['trip_speed_mph'] = speed_and_delay.trip_speed * MPH_PER_MPS

speed_and_delay['trip_speed_mph'].mean()

12.388162329259766

In [66]:
speed_and_delay.head(3)

,trip_key,arrival_time,actual_time,delay,stop_id,trip_id,shape_id,direction_id,direction,stop_sequence,route_id,shape_meters,service_date,geometry,total_delay,start_time,start_meters,seconds,trip_speed,trip_speed_mph
0,-5238867171190781897,2021-11-30 16:54:46,2021-11-30 17:01:10.276088,0 days 00:06:24.276088,4018,9296629,460067,0,Westbound,29,46,6580.90515,2021-11-30,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 16:36:57.753146,798.756471,1452,3.982196,8.908173
1,8196610795793304905,2021-11-30 18:14:46,2021-11-30 18:12:56.658216,-1 days +23:58:10.658216,4018,9296633,460067,0,Westbound,29,46,6580.90515,2021-11-30,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 17:52:27.617212,798.756471,1229,4.704759,10.524546
2,-5505830704023369186,2021-11-30 19:13:46,2021-11-30 19:15:08.641013,0 days 00:01:22.641013,4018,9296636,460067,0,Westbound,29,46,6580.90515,2021-11-30,POINT (167729.882 -470046.387),0 days 08:49:11.823428,2021-11-30 18:52:55.190148,798.756471,1333,4.337696,9.703426


In [75]:
((speed_and_delay.shape_meters - speed_and_delay.start_meters).sum()
/ speed_and_delay.seconds.sum()) * MPH_PER_MPS

11.882511619897029

In [93]:
# speed_and_delay >> mutate(total_mean_mph = (
#     (_.shape_meters - _.start_meters).sum()
#     / _.seconds.sum()) * MPH_PER_MPS)

In [85]:
test_analysis.st_geo.head(3)

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,geometry,trip_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at
0,170,0,3150,33.792724,-118.222183,VILLAGES AT CABRILLO,POINT (164664.672 -467611.924),9288591,29,07:00:00,07:00:00,None,1,0,None,None,7.127999999999999,1,2021-12-17
1,170,0,3150,33.792724,-118.222183,VILLAGES AT CABRILLO,POINT (164664.672 -467611.924),9288599,29,09:36:00,09:36:00,None,1,0,None,None,7.127999999999999,1,2021-12-17
2,170,0,3150,33.792724,-118.222183,VILLAGES AT CABRILLO,POINT (164664.672 -467611.924),9288601,29,10:20:00,10:20:00,None,1,0,None,None,7.127999999999999,1,2021-12-17


In [86]:
lbt_routelines.head(3)

,calitp_itp_id,calitp_url_number,shape_id,calitp_extracted_at,geometry
0,170,0,10069,2021-12-17,"LINESTRING (161216.993 -459406.837, 161188.049..."
1,170,0,10070,2021-12-17,"LINESTRING (165555.589 -461447.899, 165529.874..."
2,170,0,1010081,2021-12-17,"LINESTRING (178338.638 -463053.608, 178362.517..."


In [92]:
lbt_trips.head(3)

,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at
0,170,0,2021-11-30,7580361081022204604,9296678,41,0,410007,2021-09-16,2099-01-01
1,170,0,2021-11-30,4117806770041309601,9288195,191,1,1910292,2021-09-16,2099-01-01
2,170,0,2021-11-30,-2383767369974821641,9296601,46,0,460067,2021-09-16,2099-01-01


In [94]:
st_geo = test_analysis.st_geo.copy()

In [99]:
one_trip_st = st_geo >> filter(_.trip_id == '9288591')

In [100]:
trips = lbt_trips.copy() >> select(-_.calitp_extracted_at, -_.calitp_deleted_at)

In [102]:
trip_st_joined = one_trip_st >> inner_join(_, trips, on = ['calitp_itp_id', 'calitp_url_number', 'trip_id'])

In [104]:
trip_st_joined.head(3)

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,geometry,trip_id,stop_sequence,arrival_time,...,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at,service_date,trip_key,route_id,direction_id,shape_id
0,170,0,3150,33.792724,-118.222183,VILLAGES AT CABRILLO,POINT (164664.672 -467611.924),9288591,29,07:00:00,...,None,None,7.127999999999999,1,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012
1,170,0,0544,33.782963,-118.132392,LOS ALTOS & PCH NE,POINT (173001.551 -468534.279),9288591,6,06:33:00,...,None,None,1.295,1,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012
2,170,0,1688,33.789962,-118.163612,PCH & JUNIPERO NW,POINT (170094.890 -467814.760),9288591,14,06:42:04,...,None,None,3.338,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012


In [106]:
one_shape = lbt_routelines >> filter(_.shape_id == '1750012')

In [107]:
trip_st_joined['shape_meters'] = (trip_st_joined.geometry
                                .apply(lambda x: one_shape.project(x)))

In [108]:
trip_st_joined >> arrange(_.stop_sequence)

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,geometry,trip_id,stop_sequence,arrival_time,...,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at,service_date,trip_key,route_id,direction_id,shape_id,shape_meters
19,170,0,1026,33.782069,-118.114850,BEACH DR & W. CAMPUS N,POINT (174628.272 -468601.239),9288591,1,06:26:00,...,None,0.0,1,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,0.000000
24,170,0,1027,33.782130,-118.120288,BEACH DR. & DORMS N,POINT (174124.462 -468604.478),9288591,2,06:27:41,...,None,0.311,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,501.040761
14,170,0,0540,33.779528,-118.122542,BELLFLOWER & ANAHEIM RD NW,POINT (173921.407 -468897.107),9288591,3,06:29:20,...,None,0.616,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,991.109291
22,170,0,0542,33.781884,-118.128889,ANAHEIM RD. & LA PASADA NW,POINT (173328.363 -468647.513),9288591,4,06:31:36,...,None,1.035,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,1666.402436
7,170,0,0543,33.783454,-118.131228,LOS ALTOS & ANAHEIM RD. SW,POINT (173108.288 -468477.715),9288591,5,06:32:34,...,None,1.215,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,1954.947256
1,170,0,0544,33.782963,-118.132392,LOS ALTOS & PCH NE,POINT (173001.551 -468534.279),9288591,6,06:33:00,...,None,1.295,1,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,2084.087810
18,170,0,4054,33.784351,-118.134517,PCH & CLARK NE,POINT (172801.703 -468384.262),9288591,7,06:33:56,...,None,1.471,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,2366.576250
26,170,0,4055,33.785886,-118.136591,PCH & PARK SE,POINT (172606.266 -468217.848),9288591,8,06:34:47,...,None,1.63,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,2623.167090
10,170,0,1346,33.788028,-118.139601,PCH & XIMENO NW,POINT (172322.827 -467985.831),9288591,9,06:36:00,...,None,1.857,1,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,2989.232013
11,170,0,1682,33.789927,-118.146186,PCH & TERMINO NW,POINT (171708.840 -467787.228),9288591,10,06:37:58,...,None,2.335,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,3758.501108


In [109]:
trip_st_joined.columns

Index(['calitp_itp_id', 'calitp_url_number', 'stop_id', 'stop_lat', 'stop_lon',
       'stop_name', 'geometry', 'trip_id', 'stop_sequence', 'arrival_time',
       'departure_time', 'stop_headsign', 'pickup_type', 'drop_off_type',
       'continuous_pickup', 'continuous_drop_off', 'shape_dist_traveled',
       'timepoint', 'calitp_extracted_at', 'service_date', 'trip_key',
       'route_id', 'direction_id', 'shape_id', 'shape_meters'],
      dtype='object')

In [111]:
date = trip_st_joined.service_date.iloc[0]

In [116]:
date + dt.timedelta(days=1)

datetime.date(2021, 12, 1)

In [124]:
dt.datetime.strptime('06:54:51', '%H:%M:%S').time()

datetime.time(6, 54, 51)

In [125]:
dt.datetime.combine(date, dt.datetime.strptime('06:54:51', '%H:%M:%S').time())

datetime.datetime(2021, 11, 30, 6, 54, 51)

In [126]:
def fix_arrival_time(gtfs_timestring):
    '''Reformats a GTFS timestamp (which allows the hour to exceed 24 to mark service day continuity)
    to standard 24-hour time.
    '''
    split = gtfs_timestring.split(':')
    hour = int(split[0])
    extra_day = 0
    if hour >= 24:
        extra_day = 1
        split[0] = str(hour - 24)
        corrected = (':').join(split)
        return corrected.strip()
    else:
        return gtfs_timestring.strip(), extra_day

In [132]:
def gtfs_time_to_dt(df):
    date = df.service_date
    timestring, extra_day = fix_arrival_time(df.arrival_time)
    df['arrival_dt'] = dt.datetime.combine(date + dt.timedelta(days=extra_day),
                        dt.datetime.strptime(timestring, '%H:%M:%S').time())
    return df

In [133]:
trip_st_joined.apply(gtfs_time_to_dt, axis=1)

,calitp_itp_id,calitp_url_number,stop_id,stop_lat,stop_lon,stop_name,geometry,trip_id,stop_sequence,arrival_time,...,shape_dist_traveled,timepoint,calitp_extracted_at,service_date,trip_key,route_id,direction_id,shape_id,shape_meters,arrival_dt
0,170,0,3150,33.792724,-118.222183,VILLAGES AT CABRILLO,POINT (164664.672 -467611.924),9288591,29,07:00:00,...,7.127999999999999,1,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,11480.641185,2021-11-30 07:00:00
1,170,0,0544,33.782963,-118.132392,LOS ALTOS & PCH NE,POINT (173001.551 -468534.279),9288591,6,06:33:00,...,1.295,1,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,2084.087810,2021-11-30 06:33:00
2,170,0,1688,33.789962,-118.163612,PCH & JUNIPERO NW,POINT (170094.890 -467814.760),9288591,14,06:42:04,...,3.338,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,5372.532975,2021-11-30 06:42:04
3,170,0,1697,33.789955,-118.185447,PCH & ATLANTIC NW,POINT (168072.693 -467854.477),9288591,19,06:48:03,...,4.595,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,7395.343575,2021-11-30 06:48:03
4,170,0,0008,33.789935,-118.190950,PCH & LOCUST NE,POINT (167563.083 -467866.435),9288591,21,06:49:33,...,4.9110000000000005,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,7904.956548,2021-11-30 06:49:33
5,170,0,0010,33.789920,-118.196910,PCH & CHESTNUT NW,POINT (167011.137 -467878.614),9288591,23,06:51:05,...,5.254,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,8457.080327,2021-11-30 06:51:05
6,170,0,1683,33.789925,-118.152136,PCH & REDONDO NW,POINT (171157.797 -467798.209),9288591,11,06:39:22,...,2.678,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,4309.422860,2021-11-30 06:39:22
7,170,0,0543,33.783454,-118.131228,LOS ALTOS & ANAHEIM RD. SW,POINT (173108.288 -468477.715),9288591,5,06:32:34,...,1.215,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,1954.947256,2021-11-30 06:32:34
8,170,0,1686,33.789959,-118.159337,PCH & TEMPLE NW,POINT (170490.818 -467807.414),9288591,13,06:41:04,...,3.092,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,4976.550698,2021-11-30 06:41:04
9,170,0,1693,33.789965,-118.175466,PCH & LBCC NE,POINT (168997.047 -467835.625),9288591,17,06:45:14,...,4.02,0,2021-12-17,2021-11-30,-5374575435373645071,175,0,1750012,6470.689966,2021-11-30 06:45:14


In [ ]:
##test/dev
def time_at_position(df, desired_position):

    global bounding_points

    try:
        next_point = (self.progressing_positions
              >> filter(_.shape_meters > desired_position)
              >> filter(_.shape_meters == _.shape_meters.min())
             )
        prev_point = (self.progressing_positions
              >> filter(_.shape_meters < desired_position)
              >> filter(_.shape_meters == _.shape_meters.max())
             )
        bounding_points = (prev_point.append(next_point).copy().reset_index(drop=True)
                >> select(-_.secs_from_last, -_.meters_from_last, -_.speed_from_last)) ## drop in case bounding points are nonconsecutive
        secs_from_last = (bounding_points.loc[1].vehicle_timestamp - bounding_points.loc[0].vehicle_timestamp).seconds
        meters_from_last = bounding_points.loc[1].shape_meters - bounding_points.loc[0].shape_meters
        speed_from_last = meters_from_last / secs_from_last

        meters_position_to_next = bounding_points.loc[1].shape_meters - desired_position
        est_seconds_to_next = meters_position_to_next / speed_from_last
        est_td_to_next = dt.timedelta(seconds=est_seconds_to_next)
        est_dt = bounding_points.iloc[-1].vehicle_timestamp - est_td_to_next

        return est_dt
    except KeyError:
        print(f'insufficient bounding points for trip {self.trip_key}, location {desired_position}', end=': ')
        print(f'start/end of route?')
        return None